In [94]:
import csv
import difflib
import json
import os
import re
import sys

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)
sys.path.append("../..")

from src.utils.data.claims_denials import (
    get_overturn_rate,
)

INPUT_DATA_DIR = "./input_data"
OUTPUT_DATA_DIR = "./output_data"
OUTPUT_IMAGE_DIR = "./images/ma"  # Set to None if you don't want to save non-required data for external use
NO_PLOT_TITLES = False  # If True, plots have no titles. Useful for cases where one wants titles rendered separately from underlying plots.

In [95]:
# TODO: download data via script to local, if necessary
download_date = "10_20_24"
# Have to use python engine given incorrectly encoded EOF in file; c backend evidently can't handle even with on_bad_lines='warn'
df = pd.read_csv(os.path.join(INPUT_DATA_DIR, f"{download_date}/medicare_qic_partc.csv"), quoting=csv.QUOTE_MINIMAL, on_bad_lines='warn', engine='python')

Skipping line 677794: unexpected end of data


In [96]:
df.head()

,Part,Decision Date,Decision,Appeal Type,Condition,Item Service,Decision Rationale,Coverage Rules,Related Reference ID
0,Part C,01/02/2020,Unfavorable,Imaging,NaN,Mammography,We decided that the Plan does not have to pre-...,The rules say that plans must pay for a medica...,NaN
1,Part C,01/02/2020,Unfavorable,Inpatient Hospital,NaN,Acute Inpatient,We decided that the Plan does not have to pay ...,The rules say that plans must pay for a medica...,NaN
2,Part C,01/02/2020,Unfavorable,Surgery,NaN,Other - Surgery,We decided that the Plan does not have to pre-...,The rules say that plans must pay for a medica...,NaN
3,Part C,01/02/2020,Unfavorable,DME/ Orthotics,NaN,Pneumatic Compression Device,We decided that the Plan does not have to prea...,The rules say that plans must pay for a medica...,NaN
4,Part C,01/02/2020,Favorable,Practitioner Services,NaN,Injections,We decided that the Plan has to pre-approve sa...,The rules say that plans must pay for a medica...,NaN


In [97]:
# Split dates
df["decision_year"] = df["Decision Date"].apply(lambda x: int(x.split("/")[-1]))
df["decision_month"] = df["Decision Date"].apply(lambda x: int(x.split("/")[0]))
df["decision_day"] = df["Decision Date"].apply(lambda x: int(x.split("/")[1]))

# Restrict attention to years with complete data
years = [2020, 2021, 2022, 2023]
df = df[df["decision_year"].isin(years)]

In [98]:
num_appeals = len(df)
complete_overturn_rate = get_overturn_rate(df,"Decision", "Favorable") # TODO: Verify favorable means favorable to insured. Ambiguous descriptor.
overturn_rate = (df["Decision"].value_counts()['Favorable'] + df["Decision"].value_counts()['Partially Favorable']) / df["Decision"].value_counts()['Unfavorable']
print(f"Number of appeals: {num_appeals}")
print(f"Overturn rate (complete overturn): {complete_overturn_rate}")
print(f"Overturn rate (complete + partial): {overturn_rate}")

Number of appeals: 547311
Overturn rate (complete overturn): 0.04495433126686655
Overturn rate (complete + partial): 0.05243210189907008


In [99]:
df["Decision"].value_counts()

Decision
Unfavorable            520044
Favorable               24604
Partially Favorable      2663
Name: count, dtype: int64

In [119]:
def generate_summary_df(df, group_col):
    """Get df of counts and stats by groups specified by group_col.
    """
    # Group by 'date' and calculate the required statistics
    summary_df = df.groupby(group_col).agg(
        num_appeals=('Part', lambda x: len(x)),
        # num_appeals=('decision_number', 'count'),
        num_favorable=('Decision', lambda x: (x == 'Favorable').sum()),
        num_partially_favorable=('Decision', lambda x: (x == 'Partially Favorable').sum()),
        num_unfavorable=('Decision', lambda x: (x == 'Unfavorable').sum()),
        num_withdrawn=('Decision', lambda x: (x == 'Withdrawn').sum()),
        overturn_rate=('Decision', lambda x: (x == 'Favorable').sum() / len(x)),
        partial_overturn_rate=('Decision', lambda x: (x == 'Partially Favorable').sum() / len(x)),
    ).reset_index()
    return summary_df


def generate_json_summary(df):
    """Get dict of counts and stats by groups specified by group_col.
    """

    # Convert the summary DataFrame to JSON
    json_output = df.to_json(orient='records', lines=True)

    # Load JSON string to a list of dictionaries
    json_list = [json.loads(line) for line in json_output.split('\n') if line]

    return json_list

In [101]:
# Yearly summary
json_summary = generate_json_summary(generate_summary_df(df, "decision_year"))
json_summary

[{'decision_year': 2020,
  'num_appeals': 94884,
  'num_favorable': 4797,
  'num_partially_favorable': 449,
  'num_unfavorable': 89638,
  'num_withdrawn': 0,
  'overturn_rate': 0.050556469,
  'partial_overturn_rate': 0.0047320939},
 {'decision_year': 2021,
  'num_appeals': 117466,
  'num_favorable': 4734,
  'num_partially_favorable': 487,
  'num_unfavorable': 112245,
  'num_withdrawn': 0,
  'overturn_rate': 0.0403010233,
  'partial_overturn_rate': 0.0041458805},
 {'decision_year': 2022,
  'num_appeals': 150268,
  'num_favorable': 6424,
  'num_partially_favorable': 618,
  'num_unfavorable': 143226,
  'num_withdrawn': 0,
  'overturn_rate': 0.0427502862,
  'partial_overturn_rate': 0.0041126521},
 {'decision_year': 2023,
  'num_appeals': 184693,
  'num_favorable': 8649,
  'num_partially_favorable': 1109,
  'num_unfavorable': 174935,
  'num_withdrawn': 0,
  'overturn_rate': 0.0468290623,
  'partial_overturn_rate': 0.0060045589}]

In [102]:
def filter_df(df: pd.DataFrame, search_term: str, cols: list[str]):
    """Filter for sub_df matching certain pattern in any of cols.
    """
    # pattern = r"\b(?:" + re.escape(search_term)+ ")\b"
    pattern = r"(?:" + re.escape(search_term) + ")"
    mask = np.column_stack(
        [
            df[col].str.contains(pattern, na=False, flags=re.IGNORECASE)
            for col in cols
        ]
    )
    sub_df = df.loc[mask.any(axis=1)]
    return sub_df

In [103]:
# Attempt to dedupe, a bit (this is complicated in this context, only merge if a lot of overlap, print all merges)
def merge_similar_objects(data, threshold=0.8):
    merged_data = []
    merges = []

    for item in data:
        matched = False

        for merged_item in merged_data:
            similarity_ratio = difflib.SequenceMatcher(None, item['name'], merged_item['name']).ratio()

            if similarity_ratio > threshold:
                # Merge the items
                merged_item['num_appeals'] += item['num_appeals']
                merged_item['num_favorable'] += item['num_favorable']
                merged_item['num_partially_favorable'] += item['num_partially_favorable']
                merged_item['num_withdrawn'] += item['num_withdrawn']
                merged_item['num_unfavorable'] += item['num_unfavorable']
                merged_item['overturn_rate'] = merged_item["num_favorable"] / merged_item["num_appeals"]
                merged_item['partial_overturn_rate'] = merged_item["num_partially_favorable"] / merged_item["num_appeals"]
                matched = True
                merges.append([item["name"], merged_item["name"]])
                
                break

        if not matched:
            # If no match is found, add the item as is
            merged_data.append(item)

    return merged_data, merges

In [104]:
condition_df = generate_summary_df(df, group_col="Condition")

In [105]:
# Restrict to conditions appealed more than Threshold times
APPEAL_THRESHOLD = 50
sub_df = condition_df[condition_df["num_appeals"] > APPEAL_THRESHOLD]

In [106]:
# Sort DF by overturn rate.
sub_df = sub_df.sort_values(by="overturn_rate", ascending=False)

# Enforce consistent capitalization
sub_df["name"] = sub_df["Condition"].str.title()

# Convert to dict/json
json_summary = generate_json_summary(sub_df)

In [107]:
deduped_summary, merges = merge_similar_objects(json_summary)

# Retain only what we show in visualization
# Rename some columns before dumping
new_name_map = {"num_appeals": "Number of Appeals", "overturn_rate": "Overturn Rate", "partial_overturn_rate": "Partial Overturn Rate"}
for rec in deduped_summary:
    del rec["Condition"]
    del rec['num_favorable']
    del rec['num_partially_favorable']
    del rec['num_withdrawn']
    del rec['num_unfavorable']
    for key in new_name_map:
        rec[new_name_map[key]] = rec[key]
        del rec[key]

In [108]:
# Dump condition json array:
with open(os.path.join(OUTPUT_DATA_DIR, download_date, "maPartcConditions.json"), 'w') as f:
    f.write(json.dumps(deduped_summary))

In [109]:
service_df = generate_summary_df(df, group_col="Item Service")

# Restrict to services appealed more than Threshold times
APPEAL_THRESHOLD = 50
sub_df = service_df[service_df["num_appeals"] > APPEAL_THRESHOLD]

# Sort DF by overturn rate.
sub_df = sub_df.sort_values(by="overturn_rate", ascending=False)

# Enforce consistent capitalization
sub_df["name"] = sub_df["Item Service"].str.title()

# Convert to dict/json
json_summary = generate_json_summary(sub_df)


deduped_summary, merges = merge_similar_objects(json_summary)

# Rename some columns before dumping
new_name_map = {"num_appeals": "Number of Appeals", "overturn_rate": "Overturn Rate", "partial_overturn_rate": "Partial Overturn Rate"}
for rec in deduped_summary:
    del rec['Item Service']
    del rec['num_favorable']
    del rec['num_partially_favorable']
    del rec['num_withdrawn']
    del rec['num_unfavorable']
    for key in new_name_map:
        rec[new_name_map[key]] = rec[key]
        del rec[key]

In [110]:
# Dump condition json array:
with open(os.path.join(OUTPUT_DATA_DIR, download_date, "maPartcServices.json"), 'w') as f:
    f.write(json.dumps(deduped_summary))

## Part D

In [111]:
df = pd.read_csv(os.path.join(INPUT_DATA_DIR, download_date, "medicare_qic_partd.csv"))

In [112]:
df.head()

,Decision_Number,Part,Decision Date,Decision_Date_Sortable,Decision,Appeal_Type,Condition,Drug,Decision Rationale,Coverage Rules
0,QIC20-002141,Part D-Drug,01/08/2020,20200108,Unfavorable,Prescription Drug,NaN,Butalbital Acetaminophen Caffeine,Medicare rules require a Part D Plan to issue ...,Citations used in this letter come from Sectio...
1,QIC20-002144,Part D-Drug,01/06/2020,20200106,Unfavorable,Prescription Drug,NaN,Oxycontin ER,Medicare rules require a Part D Plan to issue ...,Citations used in this letter come from Sectio...
2,QIC20-002145,Part D-Drug,01/13/2020,20200113,Unfavorable,Prescription Drug,NaN,Adderall,Citations used in this letter come from Sectio...,Citations used in this letter come from Sectio...
3,QIC20-002146,Part D-Drug,01/03/2020,20200103,Unfavorable,Prescription Drug,NaN,Viokace,You asked the Plan to cover Viokace. The Plan ...,Citations used in this letter come from Sectio...
4,QIC20-002147,Part D-Drug,01/06/2020,20200106,Unfavorable,Prescription Drug,Dermatitis unspecified,Fluocinonide,Medicare rules require a Part D Plan to issue ...,Citations used in this letter come from Sectio...


In [114]:
# Split dates
df["decision_year"] = df["Decision Date"].apply(lambda x: int(x.split("/")[-1]))
df["decision_month"] = df["Decision Date"].apply(lambda x: int(x.split("/")[0]))
df["decision_day"] = df["Decision Date"].apply(lambda x: int(x.split("/")[1]))

# Restrict attention to years with complete data
years = [2020, 2021, 2022, 2023]
df = df[df["decision_year"].isin(years)]

In [117]:
num_appeals = len(df)
complete_overturn_rate = get_overturn_rate(df,"Decision", "Favorable") # TODO: Verify favorable means favorable to insured. Ambiguous descriptor.
overturn_rate = (df["Decision"].value_counts()['Favorable'] + df["Decision"].value_counts()['Partially Favorable']) / df["Decision"].value_counts()['Unfavorable']
print(f"Number of appeals: {num_appeals}")
print(f"Overturn rate (complete overturn): {complete_overturn_rate}")
print(f"Overturn rate (complete + partial): {overturn_rate}")

Number of appeals: 119093
Overturn rate (complete overturn): 0.06370651507645285
Overturn rate (complete + partial): 0.07087428401866722


In [23]:
# Yearly summary
json_summary = generate_json_summary(generate_summary_df(df, "decision_year"))
json_summary

[{'decision_year': 2020,
  'num_appeals': 23738,
  'num_favorable': 2239,
  'num_partially_favorable': 166,
  'num_unfavorable': 21333,
  'num_withdrawn': 0,
  'overturn_rate': 0.0943213413,
  'partial_overturn_rate': 0.006993007},
 {'decision_year': 2021,
  'num_appeals': 25491,
  'num_favorable': 1271,
  'num_partially_favorable': 16,
  'num_unfavorable': 24204,
  'num_withdrawn': 0,
  'overturn_rate': 0.0498607352,
  'partial_overturn_rate': 0.0006276725},
 {'decision_year': 2022,
  'num_appeals': 34771,
  'num_favorable': 1858,
  'num_partially_favorable': 54,
  'num_unfavorable': 32859,
  'num_withdrawn': 0,
  'overturn_rate': 0.053435334,
  'partial_overturn_rate': 0.0015530183},
 {'decision_year': 2023,
  'num_appeals': 29624,
  'num_favorable': 1800,
  'num_partially_favorable': 53,
  'num_unfavorable': 27771,
  'num_withdrawn': 0,
  'overturn_rate': 0.0607615447,
  'partial_overturn_rate': 0.0017890899}]

In [120]:
condition_df = generate_summary_df(df, group_col="Condition")

In [121]:
# Restrict to conditions appealed more than Threshold times
APPEAL_THRESHOLD = 50
sub_df = condition_df[condition_df["num_appeals"] > APPEAL_THRESHOLD]

In [122]:
# Sort DF by overturn rate.
sub_df = sub_df.sort_values(by="overturn_rate", ascending=False)

# Enforce consistent capitalization
sub_df["name"] = sub_df["Condition"].str.title()

# Convert to dict/json
json_summary = generate_json_summary(sub_df)

In [123]:
deduped_summary, merges = merge_similar_objects(json_summary)

# Retain only what we show in visualization
new_name_map = {"num_appeals": "Number of Appeals", "overturn_rate": "Overturn Rate", "partial_overturn_rate": "Partial Overturn Rate"}
for rec in deduped_summary:
    del rec["Condition"]
    del rec['num_favorable']
    del rec['num_partially_favorable']
    del rec['num_withdrawn']
    del rec['num_unfavorable']
    for key in new_name_map:
        rec[new_name_map[key]] = rec[key]
        del rec[key]

In [124]:
# Dump condition json array:
with open(os.path.join(OUTPUT_DATA_DIR, download_date, "maPartdConditions.json"), 'w') as f:
    f.write(json.dumps(deduped_summary))

In [125]:

service_df = generate_summary_df(df, group_col="Drug")

# Restrict to services appealed more than Threshold times
APPEAL_THRESHOLD = 50
sub_df = service_df[service_df["num_appeals"] > APPEAL_THRESHOLD]

# Sort DF by overturn rate.
sub_df = sub_df.sort_values(by="overturn_rate", ascending=False)

# Enforce consistent capitalization
sub_df["name"] = sub_df["Drug"].str.title()

# Convert to dict/json
json_summary = generate_json_summary(sub_df)


deduped_summary, merges = merge_similar_objects(json_summary)
    
# Rename some columns before dumping
new_name_map = {"num_appeals": "Number of Appeals", "overturn_rate": "Overturn Rate", "partial_overturn_rate": "Partial Overturn Rate"}
for rec in deduped_summary:
    del rec['Drug']
    del rec['num_favorable']
    del rec['num_partially_favorable']
    del rec['num_withdrawn']
    del rec['num_unfavorable']
    for key in new_name_map:
        rec[new_name_map[key]] = rec[key]
        del rec[key]

In [126]:
# Dump condition json array:
with open(os.path.join(OUTPUT_DATA_DIR, download_date, "maPartdServices.json"), 'w') as f:
    f.write(json.dumps(deduped_summary))